In [4]:
import numpy as np
import cv2
import os
import torch
import spacy
import pandas as pd
import torchvision.transforms as T

from torchvision.io.video import read_video
from tqdm.notebook import tqdm
from PIL import Image
from torch.utils.data import DataLoader,Dataset
from collections import Counter
from IPython.display import clear_output

In [5]:
!python -m spacy download ru_core_news_lg

2023-10-01 02:54:58.721795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 33.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp ./drive/MyDrive/rutube_hackathon_novosibirsk.zip ./rutube_hackathon_novosibirsk.zip

In [ ]:
!zip -FF ./rutube_hackathon_novosibirsk.zip --out ./pleasework.zip
clear_output()

In [ ]:
!unzip -qq ./pleasework.zip

In [8]:
spacy_ru = spacy.load("ru_core_news_lg")

text = "Сергей Чуприн любит hi! ОН - КАРТОфЕЛЕЛЮБ!!!"
[token.text.lower() for token in spacy_ru.tokenizer(text)]

['сергей',
 'чуприн',
 'любит',
 'hi',
 '!',
 'он',
 '-',
 'картофелелюб',
 '!',
 '!',
 '!']

In [ ]:
train_csv = pd.read_csv("./rutube_hackathon_novosibirsk/train/train.csv")

In [ ]:
train_csv["len"] = train_csv["description"].apply(lambda x: len(x.split()))

In [ ]:
train_csv.len.max()

321

In [ ]:
train_csv

,video_name,stt_name,category_name,title,description,len
0,0.mp4,0.txt,Развлечения,Правильная цена I #3,С вами Макс Климток и это шоу Правильная цена!...,53
1,1.mp4,1.txt,Спорт/Игры,Три лошадиные силы | Выпуск №2,В этом новом выпуске нас ждут не менее новые и...,46
2,2.mp4,2.txt,Блоги,Хашлама | Выпуск 4 | Силиконовый ПРЕСС Давы | ...,"Привет, это Султан и Авет! Мы опять хаваем вку...",58
3,3.mp4,3.txt,Путешествия,Прогулка по стране - Владивосток,Прогулка по Владивостоку. Самому большому горо...,42
4,4.mp4,4.txt,Искусство,Артмеханика. Выпуск 3. Татуировки + Mika Vino,Были ли татуировки на теле Николая II? Почему ...,62
...,...,...,...,...,...,...
495,495.mp4,495.txt,Юмор,Тот самый мент I Выпуск 3I Мента забирают в армию,В начале рабочего дня Максиму Константиновичу ...,56
496,496.mp4,496.txt,Лайфстайл,Сколько Стоит День? Выпуск 7. Сколько тратит в...,"Сегодня мы провели целый день с певицей, актри...",68
497,497.mp4,497.txt,Развлечения,Правильная цена I #4,С вами Макс Климток и это шоу Правильная цена!...,54
498,498.mp4,498.txt,Развлечения,Тру ДЕТЕКТОР I #6,Вы смотрите шоу с детектором лжи «Тру ДЕТЕКТОР...,93


In [9]:
class Vocabulary:
    def __init__(self,freq_threshold):
        self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}

        self.stoi = {v:k for k,v in self.itos.items()}

        self.freq_threshold = freq_threshold

    def __len__(self): return len(self.itos)

    @staticmethod
    def tokenize(text):
        return [token.text.lower() for token in spacy_ru.tokenizer(text)]

    @staticmethod
    def del_timestamps(texts):
        output = ""
        for text in texts.split("\n"):
          output += "".join(text.split("]  ")[1:]) + " "
        return " ".join(text)

    def build_vocab(self, sentence_list, texts_list):
      frequencies = Counter()
      idx = 4

      for sentence in tqdm(sentence_list):
          for word in self.tokenize(sentence):
              frequencies[word] += 1

              if frequencies[word] == self.freq_threshold:
                  self.stoi[word] = idx
                  self.itos[idx] = word
                  idx += 1

      for text in tqdm(texts_list):
          text = self.del_timestamps(text)
          for word in self.tokenize(text):
              frequencies[word] += 1

              if frequencies[word] == self.freq_threshold:
                  self.stoi[word] = idx
                  self.itos[idx] = word
                  idx += 1

    def numericalize(self,text):
        tokenized_text = self.tokenize(text)
        return [ self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text ]


In [10]:
class VideoDataset(Dataset):
    def __init__(self, root_dir, captions, video_files, texts, transform=None,freq_threshold=1, build=True):
        self.root_dir = root_dir
        self.transform = transform

        self.imgs = video_files
        self.captions = captions

        text = []
        for file in tqdm(text):
            with open(self.root_dir + "/test_stt/" + file) as cf:
                    text.append(" ".join(cf.readlines()))

        self.vocab = Vocabulary(freq_threshold)
        if build:
            self.vocab.build_vocab(self.captions, texts)


    def __len__(self):
        return len(self.imgs)

    def _read_video(self, path, frames_num=25, window=30):
        frames = []
        cap = cv2.VideoCapture(self.root_dir + "/test_video/" + path)

        fps = int(cap.get(cv2.CAP_PROP_FPS))

        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        N = length // (frames_num)

        current_frame = 1
        for i in range(length):
            ret, frame = cap.read(current_frame)
            if ret and i == current_frame and len(frames) < frames_num:
                size = 226, 226
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, size)
                frames.append(frame)
                current_frame += N


        cap.release()

        return np.array(frames)

    def __getitem__(self,idx):
        caption = self.captions[idx]
        img = self.imgs[idx]

        img = np.rollaxis(self._read_video(self.imgs[idx]), 3, 0)
        img = np.array(np.array(img) / 255, dtype=np.float32)


        caption_vec = []
        caption_vec += [self.vocab.stoi["<SOS>"]]
        caption_vec += self.vocab.numericalize(caption)
        caption_vec += [self.vocab.stoi["<EOS>"]]
        if len(caption_vec) < 300:
            for i in range(300 - len(caption_vec)):
                caption_vec.append(0)

        return torch.tensor(img), torch.tensor(caption_vec[:300])

In [11]:

data_location =  "./rutube_hackathon_novosibirsk/train"
BATCH_SIZE = 10

transforms = T.Compose([
    T.Resize(226),
    T.RandomCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])


dataset =  VideoDataset(
    root_dir=data_location,
    captions=train_csv.description.tolist(),
    texts=train_csv.stt_name.tolist(),
    video_files=train_csv.video_name,
    transform=transforms
)

data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # batch_first=False
)

vocab_size = len(dataset.vocab)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

NameError: ignored

### 3) Defining the Model Architecture

Model is seq2seq model. In the **encoder** pretrained ResNet model is used to extract the features. Decoder, is the implementation of the Bahdanau Attention Decoder. In the decoder model **LSTM cell**.

In [12]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.models as models
import torchvision.models.video as video

from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as T

In [13]:
class VideoEncoderCNN(nn.Module):
    def __init__(self):
        super(VideoEncoderCNN, self).__init__()
        self.swin_t = video.swin3d_s(pretrained=True)
        for param in self.swin_t.parameters():
            param.requires_grad_(False)

        modules = list(self.swin_t.children())[:-2]
        self.swin_t = nn.Sequential(*modules)


    def forward(self, images):
        features = self.swin_t(images)
        features = features.view(features.size(0), -1, features.size(-1))

        return features


In [14]:
class VideoAttention(nn.Module):
    def __init__(self, encoder_dim,decoder_dim,attention_dim):
        super(VideoAttention, self).__init__()

        self.attention_dim = attention_dim

        self.W = nn.Linear(decoder_dim,attention_dim)
        self.U = nn.Linear(encoder_dim,attention_dim)

        self.A = nn.Linear(attention_dim,1)




    def forward(self, features, hidden_state):
        u_hs = self.U(features)
        w_ah = self.W(hidden_state)

        combined_states = torch.tanh(u_hs + w_ah.unsqueeze(1))

        attention_scores = self.A(combined_states)
        attention_scores = attention_scores.squeeze(2)


        alpha = F.softmax(attention_scores,dim=1)

        attention_weights = features * alpha.unsqueeze(2)
        attention_weights = attention_weights.sum(dim=1)

        return alpha,attention_weights


In [15]:
class VideoDecoderRNN(nn.Module):
    def __init__(self,embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim,drop_prob=0.3):
        super().__init__()

        self.vocab_size = vocab_size
        self.attention_dim = attention_dim
        self.decoder_dim = decoder_dim

        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.attention = VideoAttention(encoder_dim,decoder_dim,attention_dim)


        self.init_h = nn.Linear(encoder_dim, decoder_dim)
        self.init_c = nn.Linear(encoder_dim, decoder_dim)
        self.lstm_cell = nn.LSTMCell(embed_size+encoder_dim,decoder_dim,bias=True)
        self.f_beta = nn.Linear(decoder_dim, encoder_dim)


        self.fcn = nn.Linear(decoder_dim,vocab_size)
        self.drop = nn.Dropout(drop_prob)



    def forward(self, features, captions):
        embeds = self.embedding(captions)

        h, c = self.init_hidden_state(features)

        seq_length = len(captions[0])-1
        batch_size = captions.size(0)
        num_features = features.size(1)

        preds = torch.zeros(batch_size, seq_length, self.vocab_size).to(device)
        alphas = torch.zeros(batch_size, seq_length,num_features).to(device)

        for s in range(seq_length):
            alpha,context = self.attention(features, h)
            lstm_input = torch.cat((embeds[:, s], context), dim=1)
            h, c = self.lstm_cell(lstm_input, (h, c))

            output = self.fcn(self.drop(h))

            preds[:,s] = output
            alphas[:,s] = alpha


        return preds, alphas

    def generate_caption(self,features, max_len=300 ,vocab=None):

        batch_size = features.size(0)
        h, c = self.init_hidden_state(features)

        alphas = []
        word = torch.tensor(vocab.stoi['<SOS>']).view(1,-1).to(device)
        embeds = self.embedding(word)


        captions = []

        for i in range(max_len):
            alpha,context = self.attention(features, h)

            alphas.append(alpha.cpu().detach().numpy())

            lstm_input = torch.cat((embeds[:, 0], context), dim=1)
            h, c = self.lstm_cell(lstm_input, (h, c))
            output = self.fcn(self.drop(h))
            output = output.view(batch_size,-1)


            predicted_word_idx = output.argmax(dim=1)

            captions.append(predicted_word_idx.item())

            if vocab.itos[predicted_word_idx.item()] == "<EOS>":
                break

            embeds = self.embedding(predicted_word_idx.unsqueeze(0))

        return [vocab.itos[idx] for idx in captions],alphas


    def init_hidden_state(self, encoder_out):
        mean_encoder_out = encoder_out.mean(dim=1)
        h = self.init_h(mean_encoder_out)
        c = self.init_c(mean_encoder_out)
        return h, c


In [16]:
class VideoEncoderDecoder(nn.Module):
    def __init__(self,embed_size, vocab_size, attention_dim,encoder_dim,decoder_dim,drop_prob=0.3):
        super().__init__()
        self.encoder = VideoEncoderCNN()
        self.decoder = VideoDecoderRNN(
            embed_size=embed_size,
            vocab_size = vocab_size,
            attention_dim=attention_dim,
            encoder_dim=encoder_dim,
            decoder_dim=decoder_dim
        )

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs = self.decoder(features, captions)
        return outputs


### 4) Setting Hypperparameter and Init the model

In [17]:
#Hyperparams
embed_size=300
vocab_size = len(dataset.vocab)
attention_dim=256
encoder_dim=768
decoder_dim=512
learning_rate = 3e-4


NameError: ignored

In [ ]:
#init model
model = VideoEncoderDecoder(
    embed_size=embed_size,
    vocab_size=vocab_size,
    attention_dim=attention_dim,
    encoder_dim=encoder_dim,
    decoder_dim=decoder_dim
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Swin3D_S_Weights.KINETICS400_V1`. You can also use `weights=Swin3D_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# !pip install av

In [ ]:
#helper function to save the model
def save_model(model,num_epochs):
    model_state = {
        'num_epochs':num_epochs,
        'embed_size':embed_size,
        'vocab_size':len(dataset.vocab),
        'attention_dim':attention_dim,
        'encoder_dim':encoder_dim,
        'decoder_dim':decoder_dim,
        'state_dict':model.state_dict()
    }

    torch.save(model_state,'./drive/MyDrive/attention_model_state.pth')

## 5) Training Job from above configs

In [18]:
torch.cuda.empty_cache()

In [19]:
import nltk
from nltk.translate import meteor
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
def score(text, text_sum):
    text = text.replace("<PAD>", "")
    if isinstance(text_sum, str):
        return round(meteor([word_tokenize(text)],word_tokenize(text_sum)), 4)
    else:
        return 0

In [ ]:
num_epochs = 25
print_every = 10

for epoch in tqdm(range(1, num_epochs + 1)):
    losses = []
    print(f"Epoch: {epoch}")
    try:
        for idx, (image, captions) in tqdm(enumerate(data_loader), total=len(data_loader)):
            # print(image.shape[2])
            # if image.shape[2] < 50:
            #     continue
            image, captions = image.to(device),captions.to(device)

            optimizer.zero_grad()

            outputs,attentions = model(image, captions)

            loss = criterion(outputs.view(-1, vocab_size), captions[:, 1:].reshape(-1))

            loss.backward()

            optimizer.step()
            losses.append(loss.item())

            if idx % print_every == 0:
                model.eval()
                with torch.no_grad():
                    dataiter = iter(data_loader)
                    img,targ = next(dataiter)
                    features = model.encoder(img[0:1].to(device))
                    caps, alphas = model.decoder.generate_caption(features, vocab=dataset.vocab)
                    caption = ' '.join(caps)
                    target = " ".join([dataset.vocab.itos[i] for i in targ[0:1][0][1:].tolist()])
                    print("true:", target)
                    print("pred:", caption)
                    print("loss:", loss.item(), "score:", score(target, caption))


                model.train()
    except Exception as ex:
        print("Err:", ex)
        raise ex
    if len(losses) != 0:
        print("Epoch loss: {:.5f}".format(sum(losses) / len(losses)))
    else:
        print(losses)
    #save the latest model
    save_model(model,epoch)
    # clear_output(wait=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 1


  0%|          | 0/50 [00:00<?, ?it/s]

loss: 8.915692329406738 score: 0.0263
loss: 8.47883129119873 score: 0.0227
loss: 7.248097896575928 score: 0.0176
loss: 7.123484134674072 score: 0.0221
loss: 7.134592533111572 score: 0.0329
Epoch loss: 7.71725
Epoch: 2


  0%|          | 0/50 [00:00<?, ?it/s]

...

loss: 2.7467591762542725 score: 0.467
loss: 2.439303159713745 score: 0.086
loss: 1.9384729862213135 score: 0.3129
loss: 2.513373374938965 score: 0.0873
loss: 2.6745805740356445 score: 0.2657
Epoch loss: 2.47120
Epoch: 24


  0%|          | 0/50 [00:00<?, ?it/s]

loss: 2.4338889122009277 score: 0.2219
loss: 1.7518889904022217 score: 0.3628
loss: 2.1144773960113525 score: 0.7563
loss: 2.2498204708099365 score: 0.7166
loss: 2.5680336952209473 score: 0.2869
Epoch loss: 2.35807
Epoch: 25


  0%|          | 0/50 [00:00<?, ?it/s]

loss: 1.9401556253433228 score: 0.1478
loss: 2.2931458950042725 score: 0.6116
loss: 2.0408644676208496 score: 0.1866
loss: 2.161356210708618 score: 0.1696
loss: 1.8561944961547852 score: 0.5481
Epoch loss: 2.21570


In [49]:
checkpoint = torch.load("./drive/MyDrive/video_model.pth")

In [50]:
model = VideoEncoderDecoder(
    embed_size=checkpoint["embed_size"],
    vocab_size =checkpoint["vocab_size"],
    attention_dim=checkpoint["attention_dim"],
    encoder_dim=checkpoint["encoder_dim"],
    decoder_dim=checkpoint["decoder_dim"]
).to(device)
model.load_state_dict(checkpoint["state_dict"])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Swin3D_S_Weights.KINETICS400_V1`. You can also use `weights=Swin3D_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [21]:
!wget https://lodmedia.hb.bizmrg.com/case_files/1026506/test_dataset_test.zip

--2023-10-01 02:59:53--  https://lodmedia.hb.bizmrg.com/case_files/1026506/test_dataset_test.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 95.163.53.117
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|95.163.53.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1696238228 (1.6G) [application/zip]
Saving to: ‘test_dataset_test.zip’

test_dataset_test.z 100%[===================>]   1.58G  48.0MB/s    in 50s     

2023-10-01 03:00:44 (32.5 MB/s) - ‘test_dataset_test.zip’ saved [1696238228/1696238228]



In [51]:
import pickle

In [22]:
!unzip -qq test_dataset_test.zip

In [52]:
train_csv = pd.read_csv("./test/test.csv")

In [53]:
sample_submission = pd.read_csv("./sample_submission.csv")

In [54]:
sample_submission.head()

,video_name,generated_description
0,0.mp4,Полученное описание
1,1.mp4,Полученное описание
2,2.mp4,Полученное описание
3,3.mp4,Полученное описание
4,4.mp4,Полученное описание


In [55]:

data_location =  "./test/"
BATCH_SIZE = 1

transforms = T.Compose([
    T.Resize(226),
    T.RandomCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])


dataset =  VideoDataset(
    root_dir=data_location,
    captions=train_csv.video_name.tolist(),
    texts=train_csv.stt_name.tolist(),
    video_files=train_csv.video_name,
    transform=transforms,
    build=False
)

dataset.vocab = pickle.load(open("./vocab.pkl", "rb"))


data_loader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    # batch_first=False
)

vocab_size = len(dataset.vocab)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

0it [00:00, ?it/s]

device(type='cuda', index=0)

In [56]:
import string

In [57]:
scores = []
model.eval()

for idx, (image, captions) in tqdm(enumerate(data_loader), total=len(data_loader)):
    img, targ = image.to(device),captions.to(device)
    with torch.no_grad():
        features = model.encoder(img[0:1].to(device))
        caps, alphas = model.decoder.generate_caption(features, vocab=dataset.vocab)
        caption = ' '.join(caps)
        caption = caption.replace("<EOS>", "")
        for i in string.punctuation:
            if i not in ["/", "\\", "-"]:
                caption = caption.replace(f" {i}", i)
        # print(targ, caption)
        sample_submission.loc[sample_submission.video_name == f"{idx}.mp4", "generated_description"] = [caption]
        print(caption)
        # target = " ".join([dataset.vocab.itos[i] for i in targ[0:1][0][1:].tolist()])

        # scores.append(score(target, caption))
        # print("true:", target)
        # print("pred:", caption)
        # print("loss:", loss.item(), "score:", score(target, caption))


  0%|          | 0/100 [00:00<?, ?it/s]

добро пожаловать на канал « бюро добрых дел »! его глава – это я, лариса брохман. а миссия нашего бюро – искать самые добрые и хорошие новости по всему миру, чтобы рассказать их вам. в этом выпуске:   - /   - /   - /   - /   - /   ссылка на email   в этом выпуске:   - /   - /   - /   в этом выпуске - /   гость - специальный гость шоу « пацанский клининг », чтобы рассказать их вам. в этом выпуске: -   гость - ксения егорова в рамках рубрики « добрый репортаж », чтобы поднять вам настроение сделать ваш день чуточку радостнее и светлее.   пишите свои добрые истории нам на почту: email   по вопросам рекламы пишите на email 
знаете, как в котором комики играют и не менее от шефа. 
вы смотрите шоу с детектором лжи « тру детектор »! шоу, в котором кристи крайм и её молчаливый друг приглашают в гости известных людей и задают им неудобные вопросы под пристальным взором полиграфа. в этом выпуске, что скрывают звёзды и их ближайшее окружение? какие вопросы они зададут друг другу? и кто окажется с

In [58]:
sample_submission.head()

,video_name,generated_description
0,0.mp4,добро пожаловать на канал « бюро добрых дел »!...
1,1.mp4,"знаете, как в котором комики играют и не менее..."
2,2.mp4,вы смотрите шоу с детектором лжи « тру детекто...
3,3.mp4,в этом выпуске: антон протеинов - / - / в ...
4,4.mp4,в этом выпуске ведущая алиса и дизайнер наталь...
